In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
p = original + 0.001*original2
u2, s2, vh2 = np.linalg.svd(p)

NameError: name 'original' is not defined

In [ ]:
np.argmax(np.dot(u[:,3], u2))

In [ ]:
# Define a matrix, mask and noise
p = 0.75
rank = 1

n = 4

U = np.random.randn(n, rank)
V = np.random.randn(n, rank)
original = np.dot(U, V.T)
mask = np.random.choice([0, 1], size=(n,n), p=[1-p, p])

In [ ]:
def minimize_singular_value(sing_val_index):

    starting_point = np.copy(original)

    boolMask = np.ma.make_mask(np.where(np.array(mask) < 0.5, 1, 0))
    starting_point[boolMask] = 0

    prev_norm = np.linalg.norm(starting_point,'nuc')

    current_point = starting_point

    def obj_func(m, vec):
        u, s, vh = np.linalg.svd(m)
        index = np.argmax(np.dot(vec, u))

        return s[index]

    def comp_grad(of, m, vec):
        """ Computes gradient that maximizes the objective function """
        epsilon = 1e-3

        grad = np.zeros([n,n])

        for i in range(n):
            for j in range(n):
                if boolMask[i,j]:
                    diff = np.zeros([n,n])
                    diff[i,j] = epsilon
                    grad[i,j] = (of(m + diff, vec) - of(m - diff, vec))/(2*epsilon)

        return grad


    u, s, vh = np.linalg.svd(current_point)
    vec = u[:,sing_val_index]

    sing_val_log = []
    distances_log = []
    
    for i in range(2000):
        u, s, vh = np.linalg.svd(current_point)
        index = np.argmax(np.dot(vec, u))
        vec = u[:,index]

        print(i, index, s[index], np.linalg.norm(current_point-original,'fro'))
        sing_val_log.append(s[index])
        distances_log.append(np.linalg.norm(current_point-original,'fro'))
        current_point = current_point - 0.1*comp_grad(obj_func, current_point, vec)
    
    return current_point, sing_val_log, distances_log

In [ ]:
result, sing_val_log, distances_log = minimize_singular_value(2)

### A bit of mess

In [ ]:
starting_point = np.copy(original)

boolMask = np.ma.make_mask(np.where(np.array(mask) < 0.5, 1, 0))
starting_point[boolMask] = 0

prev_norm = np.linalg.norm(starting_point,'nuc')

current_point = starting_point

def obj_func(m, vec):
    u, s, vh = np.linalg.svd(m)
    index = np.argmax(np.dot(vec, u))

    return s[index]

def comp_grad(of, m, vec):
    """ Computes gradient that maximizes the objective function """
    epsilon = 1e-3

    grad = np.zeros([n,n])

    for i in range(n):
        for j in range(n):
            if boolMask[i,j]:
                diff = np.zeros([n,n])
                diff[i,j] = epsilon
                grad[i,j] = (of(m + diff, vec) - of(m - diff, vec))/(2*epsilon)

    return grad

def line_search(point, grad, obj_func, vec, s = 0.01, threshold = 1e-10):
    """ Finds a maximum step size that maximizes the descent (since computing gradient is hard) """

    choice = point - grad * s
    while obj_func(choice, vec) > obj_func(point, vec):
        s = s / 2
        choice = point - grad * s

    initial_s = s

    while True:
        left = choice + grad * s/2
        right = point - grad * s/2

        s = s / 2
        if obj_func(left, vec) < obj_func(right, vec):
            choice = left
        else:
            choice = right

        if np.abs(obj_func(left,vec) - obj_func(right, vec)) < threshold:
            break

    return choice

sing_val_log = []
distances_log = []

for sing_val_index in range(3,0,-1):

    print("\n")
    print("##################")
    print("Optimizing", sing_val_index)
    print("##################")
    print("\n")
    
    u, s, vh = np.linalg.svd(current_point)
    vec = u[:,sing_val_index]

    i = 0
    while obj_func(current_point, vec) > 0.08:
        i = i + 1
        u, s, vh = np.linalg.svd(current_point)
        index = np.argmax(np.dot(vec, u))
        vec = u[:,index]

        print(i, index, s[index], np.linalg.norm(current_point-original,'fro'))
        sing_val_log.append(s[index])
        distances_log.append(np.linalg.norm(current_point-original,'fro'))
        #current_point = current_point - 0.0001*comp_grad(obj_func, current_point, vec)
        grad = comp_grad(obj_func, current_point, vec)
        current_point = line_search(current_point, grad, obj_func, vec)
    

In [ ]:
plt.plot(sing_val_log)

In [ ]:
plt.plot(distances_log)

TODO:

* use projected gradient descent